In [14]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from generative_social_choice.datasets.datasets import get_dataset
from generative_social_choice.objects.agents import CardinalAgent
from generative_social_choice.objects.committee import generate_slate
from generative_social_choice.objects.moderators import (
    MultiLevelQuery1Moderator,
    PolarizeModerator,
)
from generative_social_choice.experiments.prolific_approval_query_eval.create_agents import (
    get_cardinal_fewshot_agents_without_test,
    get_choices,
)
from generative_social_choice.utils.gpt_wrapper import prompt_gpt

Decide what choices to use for Query2

In [16]:
choices = get_choices("chatbot_personalization_rating_23_09_25")

In [17]:
choices

['This statement does not reflect my opinion at all.',
 'I would be satisfied with substantial changes.',
 'I would be satisfied with minor changes.',
 "I am completely satisfied, there is nothing I'd like change."]

In [18]:
# Sara's attempt at just getting more fine grained stuff
choices = [
    "This statement is garbage, it's completely opposite to my opinions.",
    "This statement does not reflect my opinions at all.",
    "I mostly disagree with the statement, but it has a little bit that rings true.",
    "Half of the statement seems fine, the other half I disagree with.",
    "I mostly agree, but I'd want to make quite a few changes to the statement.",
    "I think this statement mostly has it right, but it's missing this aspect of my opinion.",
    "This statement almost precisely mirrors my opinion, it's just slightly off.",
    "This perfectly encapsulates what I believe on this topic, just in different words.",
]

Get agents from `experiment1` and just use their comments

In [19]:
dataset = get_dataset("prolific")
df = dataset.load()
experiment_type_to_agent_ids = dataset.get_agent_ids()
agent_ids = experiment_type_to_agent_ids["experiment1"]
agent_ids_to_comment = {
    agent_id: dataset.get_transcript_from_agent_id(agent_id)[
        dataset.get_transcript_from_agent_id(agent_id)["step"] == 7
    ]["text"].values[0]
    for agent_id in agent_ids
}

approval_level = 5
agents = [
    CardinalAgent(
        id=agent_id, prompt_type="cot", agent_opinion=comment, choices=choices
    )
    for agent_id, comment in agent_ids_to_comment.items()
]

In [7]:
print(agents[0].prompt_template)

In a moment I will give you opinions a user has expressed on a topic. Your job is to determine what the user's opinion on a specific statement would be. Specifically, you should select one of the following choices:
(1) This statement is garbage, it's completely opposite to my opinions.
(2) This statement does not reflect my opinions at all.
(3) I mostly disagree with the statement, but it has a little bit that rings true.
(4) Half of the statement seems fine, the other half I disagree with.
(5) I mostly agree, but I'd want to make quite a few changes to the statement.
(6) I think this statement mostly has it right, but it's missing this aspect of my opinion.
(7) This statement almost precisely mirrors my opinion, it's just slightly off.
(8) This perfectly encapsulates what I believe on this topic, just in different words.


Now I will give you the opinions the first user wrote: I think it would save time and be faster if it remembered you. I believe knowing your personal preferences wo

In [11]:
n = len(agents)

In [12]:
k = 5

In [13]:
assert n % k == 0

Instead get agents from the latest 20230927 data, with both multi level approavl voes and agent_opinions

In [8]:
agents = get_cardinal_fewshot_agents_without_test(
    experiment_name="chatbot_personalization_rating_23_09_27"
)

Pick moderator

In [11]:
approval_level = 5
moderator = MultiLevelQuery1Moderator(
    id=0, prompt_type="basic_cot", approval_level=approval_level
)

In [10]:
moderator = PolarizeModerator(
    id=0, prompt_type="fixed_cluster_size_cot", cluster_size=n // k
)

Make JR slate

In [11]:
committee_supporters = generate_slate(
    algorithm_type="BalancedJR",
    agents=agents,
    query1_moderator=moderator,
    query1_prime_moderator=None,
    k=k,
    experiment_type="4app_level_new_data_polarize",
    log_dir_name="experiments/openai_jr_demo/",
)

Running balanced JR...
Step 1: (current committee {})
Query 1 response: Chatbot personalization should be limited and user-controlled, with a focus on maintaining privacy and avoiding overpersonalization that may compromise accuracy and user experience.
Testing agent 56588a127d69570012dfe00a (comment=They should remember all of the info you have put in so it can fully understand the tasks you want from it so that you don't have to repeat yourself.  The more info it learns about you and your situation, the more it will help.)
Response: 3
Testing agent 5c93e410ec114b001641a170 (comment=I fully agree that chatbots should be personalized when conversing with users. Not completely enough to compromise their ability to answer questions, but just a smidge to spice up conversations. Others might say that this is dangerous since it could influence naive users, but I feel like with enough programming we could mitigate this so it could work.)
Response: 3
Testing agent 63483ed07567f8b5238da77c (co